<a href="https://colab.research.google.com/github/kyochanpy/Kaggle_Indoor_Location_Navigation/blob/main/createa_dataset/features_wifi_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import glob
import os
import gc
import json 

In [3]:
base_path = '/content/drive/MyDrive/fixed_train'

In [4]:
# pull out all the buildings actually used in the test set, given current method we don't need the other ones
ssubm = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')

# only 24 of the total buildings are used in the test set, 
# this allows us to greatly reduce the intial size of the dataset

ssubm_df = ssubm["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))
used_buildings = sorted(ssubm_df[0].value_counts().index.tolist())

# dictionary used to map the floor codes to the values used in the submission file. 
floor_map = {"B2":-2, "B1":-1, "F1":0, "F2": 1, "F3":2, "F4":3, "F5":4, "F6":5, "F7":6,"F8":7, "F9":8,
             "1F":0, "2F":1, "3F":2, "4F":3, "5F":4, "6F":5, "7F":6, "8F": 7, "9F":8}

In [5]:
# get only the wifi bssid that occur over 1000 times(this number can be experimented with)
# these will be the only ones used when constructing features
bssid = dict()

for building in used_buildings:
    folders = sorted(glob.glob(os.path.join(base_path,'train/'+building+'/*')))
    print(building)
    wifi = list()
    for folder in folders:
        floor = floor_map[folder.split('/')[-1]]
        files = glob.glob(os.path.join(folder, "*.txt"))
        for file in files:
            with open(file) as f:
                txt = f.readlines()
                for e, line in enumerate(txt):
                    tmp = line.strip().split()
                    if tmp[1] == "TYPE_WIFI":
                        wifi.append(tmp)
    df = pd.DataFrame(wifi)
    #top_bssid = df[3].value_counts().iloc[:500].index.tolist()
    value_counts = df[3].value_counts()
    top_bssid = value_counts[value_counts > 0].index.tolist()
    print(len(top_bssid))
    bssid[building] = top_bssid
    del df
    del wifi
    gc.collect()

5a0546857ecc773753327266
3397
5c3c44b80379370013e0fd2b
3063
5d27075f03f801723c2e360f
7029
5d27096c03f801723c31e5e0
4964
5d27097f03f801723c320d97
2490
5d27099f03f801723c32511d
925
5d2709a003f801723c3251bf
1252
5d2709b303f801723c327472
1913
5d2709bb03f801723c32852c
2452
5d2709c303f801723c3299ee
5831
5d2709d403f801723c32bd39
2139
5d2709e003f801723c32d896
1309
5da138274db8ce0c98bbd3d2
490
5da1382d4db8ce0c98bbe92e
2862
5da138314db8ce0c98bbf3a0
1212
5da138364db8ce0c98bc00f1
822
5da1383b4db8ce0c98bc11ab
1525
5da138754db8ce0c98bca82f
1627
5da138764db8ce0c98bcaa46
1888
5da1389e4db8ce0c98bd0547
1021
5da138b74db8ce0c98bd4774
3535
5da958dd46f8266d0737457b
3499
5dbc1d84c1eb61796cf7c010
4519
5dc8cea7659e181adb076a3f
4864


In [6]:
with open("bssid_0.json", "w") as f:
    json.dump(bssid, f)

with open("bssid_0.json") as f:
    bssid = json.load(f)

In [7]:
# generate all the training data 
building_dfs = dict()

for building in used_buildings:
    folders = sorted(glob.glob(os.path.join(base_path,'train', building +'/*')))
    dfs = list()
    index = sorted(bssid[building])
    print(building)
    for folder in folders:
        floor = floor_map[folder.split('/')[-1]]
        files = glob.glob(os.path.join(folder, "*.txt"))
        print(floor)
        for file in files:
            wifi = list()
            waypoint = list()
            with open(file) as f:
                txt = f.readlines()
            for line in txt:
                line = line.strip().split()
                if line[1] == "TYPE_WAYPOINT":
                    waypoint.append(line)
                if line[1] == "TYPE_WIFI":
                    wifi.append(line)

            df = pd.DataFrame(np.array(wifi))    

            # generate a feature, and label for each wifi block
            for gid, g in df.groupby(0):
                dists = list()
                for e, k in enumerate(waypoint):
                    dist = abs(int(gid) - int(k[0]))
                    dists.append(dist)
                nearest_wp_index = np.argmin(dists)
                
                g = g.drop_duplicates(subset=3)
                tmp = g.iloc[:,3:5]
                feat = tmp.set_index(3).reindex(index).replace(np.nan, -999).T
                feat["x"] = float(waypoint[nearest_wp_index][2])
                feat["y"] = float(waypoint[nearest_wp_index][3])
                feat["f"] = floor
                feat["path"] = file.split('/')[-1].split('.')[0] # useful for crossvalidation
                dfs.append(feat)
                
    building_df = pd.concat(dfs)
    building_dfs[building] = df
    building_df.to_csv(building+"_0_train.csv")

5a0546857ecc773753327266
-1
0
1
2
3
5c3c44b80379370013e0fd2b
-1
0
1
2
3
4
5d27075f03f801723c2e360f
-1
0
1
2
3
4
5
6
5d27096c03f801723c31e5e0
-1
0
1
2
3
4
5
5d27097f03f801723c320d97
-1
-2
0
1
2
3
4
5d27099f03f801723c32511d
-1
0
1
2
3
5d2709a003f801723c3251bf
0
1
2
3
5d2709b303f801723c327472
0
1
2
3
-1
5d2709bb03f801723c32852c
-1
0
1
2
3
5d2709c303f801723c3299ee
0
1
2
3
4
5
6
7
8
-1
5d2709d403f801723c32bd39
0
1
2
-1
5d2709e003f801723c32d896
-1
0
1
2
3
4
5da138274db8ce0c98bbd3d2
0
1
2
5da1382d4db8ce0c98bbe92e
-1
0
1
2
3
4
5da138314db8ce0c98bbf3a0
-1
-2
0
1
2
5da138364db8ce0c98bc00f1
0
1
2
5da1383b4db8ce0c98bc11ab
0
1
2
5da138754db8ce0c98bca82f
0
1
2
3
5da138764db8ce0c98bcaa46
-1
0
1
2
3
4
5da1389e4db8ce0c98bd0547
-1
-2
0
1
2
3
5da138b74db8ce0c98bd4774
-1
-2
0
1
2
3
4
5da958dd46f8266d0737457b
-1
0
1
2
3
4
5
6
5dbc1d84c1eb61796cf7c010
-1
1
2
3
4
5
6
7
5dc8cea7659e181adb076a3f
-1
0
1
2
3
4
5
6


In [8]:
# Generate the features for the test set

ssubm_building_g = ssubm_df.groupby(0)
feature_dict = dict()

for gid0, g0 in ssubm_building_g:
    index = sorted(bssid[g0.iloc[0,0]])
    feats = list()
    print(gid0)
    for gid,g in g0.groupby(1):

        # get all wifi time locations, 
        with open(os.path.join(base_path, 'test/' + g.iloc[0,1] + '.txt')) as f:
            txt = f.readlines()

        wifi = list()

        for line in txt:
            line = line.strip().split()
            if line[1] == "TYPE_WIFI":
                wifi.append(line)

        wifi_df = pd.DataFrame(wifi)
        wifi_points = pd.DataFrame(wifi_df.groupby(0).count().index.tolist())
        
        for timepoint in g.iloc[:,2].tolist():

            deltas = (wifi_points.astype(int) - int(timepoint)).abs()
            min_delta_idx = deltas.values.argmin()
            wifi_block_timestamp = wifi_points.iloc[min_delta_idx].values[0]
            
            wifi_block = wifi_df[wifi_df[0] == wifi_block_timestamp].drop_duplicates(subset=3)
            feat = wifi_block.set_index(3)[4].reindex(index).fillna(-999)

            feat['site_path_timestamp'] = g.iloc[0,0] + "_" + g.iloc[0,1] + "_" + timepoint
            feats.append(feat)
    feature_df = pd.concat(feats, axis=1).T
    feature_df.to_csv(gid0+"_0_test.csv")
    feature_dict[gid0] = feature_df

5a0546857ecc773753327266
5c3c44b80379370013e0fd2b
5d27075f03f801723c2e360f
5d27096c03f801723c31e5e0
5d27097f03f801723c320d97
5d27099f03f801723c32511d
5d2709a003f801723c3251bf
5d2709b303f801723c327472
5d2709bb03f801723c32852c
5d2709c303f801723c3299ee
5d2709d403f801723c32bd39
5d2709e003f801723c32d896
5da138274db8ce0c98bbd3d2
5da1382d4db8ce0c98bbe92e
5da138314db8ce0c98bbf3a0
5da138364db8ce0c98bc00f1
5da1383b4db8ce0c98bc11ab
5da138754db8ce0c98bca82f
5da138764db8ce0c98bcaa46
5da1389e4db8ce0c98bd0547
5da138b74db8ce0c98bd4774
5da958dd46f8266d0737457b
5dbc1d84c1eb61796cf7c010
5dc8cea7659e181adb076a3f


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
!mv /content/5a0546857ecc773753327266_0_train.csv /content/drive/MyDrive/wifi_0
!mv /content/5c3c44b80379370013e0fd2b_0_train.csv /content/drive/MyDrive/wifi_0
!mv /content/5d27075f03f801723c2e360f_0_train.csv /content/drive/MyDrive/wifi_0
!mv /content/5d27096c03f801723c31e5e0_0_train.csv /content/drive/MyDrive/wifi_0
!mv /content/5d27097f03f801723c320d97_0_train.csv /content/drive/MyDrive/wifi_0
!mv /content/5d27099f03f801723c32511d_0_train.csv /content/drive/MyDrive/wifi_0
!mv /content/5d2709a003f801723c3251bf_0_train.csv /content/drive/MyDrive/wifi_0
!mv /content/5d2709b303f801723c327472_0_train.csv /content/drive/MyDrive/wifi_0
!mv /content/5d2709bb03f801723c32852c_0_train.csv /content/drive/MyDrive/wifi_0
!mv /content/5d2709c303f801723c3299ee_0_train.csv /content/drive/MyDrive/wifi_0
!mv /content/5d2709d403f801723c32bd39_0_train.csv /content/drive/MyDrive/wifi_0
!mv /content/5d2709e003f801723c32d896_0_train.csv /content/drive/MyDrive/wifi_0
!mv /content/5da138274db8ce0c98bbd3d2_0_train.csv /content/drive/MyDrive/wifi_0
!mv /content/5da1382d4db8ce0c98bbe92e_0_train.csv /content/drive/MyDrive/wifi_0
!mv /content/5da138314db8ce0c98bbf3a0_0_train.csv /content/drive/MyDrive/wifi_0
!mv /content/5da138364db8ce0c98bc00f1_0_train.csv /content/drive/MyDrive/wifi_0
!mv /content/5da1383b4db8ce0c98bc11ab_0_train.csv /content/drive/MyDrive/wifi_0
!mv /content/5da138754db8ce0c98bca82f_0_train.csv /content/drive/MyDrive/wifi_0
!mv /content/5da138764db8ce0c98bcaa46_0_train.csv /content/drive/MyDrive/wifi_0
!mv /content/5da1389e4db8ce0c98bd0547_0_train.csv /content/drive/MyDrive/wifi_0
!mv /content/5da138b74db8ce0c98bd4774_0_train.csv /content/drive/MyDrive/wifi_0
!mv /content/5da958dd46f8266d0737457b_0_train.csv /content/drive/MyDrive/wifi_0
!mv /content/5dbc1d84c1eb61796cf7c010_0_train.csv /content/drive/MyDrive/wifi_0
!mv /content/5dc8cea7659e181adb076a3f_0_train.csv /content/drive/MyDrive/wifi_0

In [10]:
!mv /content/5a0546857ecc773753327266_0_test.csv /content/drive/MyDrive/wifi_0
!mv /content/5c3c44b80379370013e0fd2b_0_test.csv /content/drive/MyDrive/wifi_0
!mv /content/5d27075f03f801723c2e360f_0_test.csv /content/drive/MyDrive/wifi_0
!mv /content/5d27096c03f801723c31e5e0_0_test.csv /content/drive/MyDrive/wifi_0
!mv /content/5d27097f03f801723c320d97_0_test.csv /content/drive/MyDrive/wifi_0
!mv /content/5d27099f03f801723c32511d_0_test.csv /content/drive/MyDrive/wifi_0
!mv /content/5d2709a003f801723c3251bf_0_test.csv /content/drive/MyDrive/wifi_0
!mv /content/5d2709b303f801723c327472_0_test.csv /content/drive/MyDrive/wifi_0
!mv /content/5d2709bb03f801723c32852c_0_test.csv /content/drive/MyDrive/wifi_0
!mv /content/5d2709c303f801723c3299ee_0_test.csv /content/drive/MyDrive/wifi_0
!mv /content/5d2709d403f801723c32bd39_0_test.csv /content/drive/MyDrive/wifi_0
!mv /content/5d2709e003f801723c32d896_0_test.csv /content/drive/MyDrive/wifi_0
!mv /content/5da138274db8ce0c98bbd3d2_0_test.csv /content/drive/MyDrive/wifi_0
!mv /content/5da1382d4db8ce0c98bbe92e_0_test.csv /content/drive/MyDrive/wifi_0
!mv /content/5da138314db8ce0c98bbf3a0_0_test.csv /content/drive/MyDrive/wifi_0
!mv /content/5da138364db8ce0c98bc00f1_0_test.csv /content/drive/MyDrive/wifi_0
!mv /content/5da1383b4db8ce0c98bc11ab_0_test.csv /content/drive/MyDrive/wifi_0
!mv /content/5da138754db8ce0c98bca82f_0_test.csv /content/drive/MyDrive/wifi_0
!mv /content/5da138764db8ce0c98bcaa46_0_test.csv /content/drive/MyDrive/wifi_0
!mv /content/5da1389e4db8ce0c98bd0547_0_test.csv /content/drive/MyDrive/wifi_0
!mv /content/5da138b74db8ce0c98bd4774_0_test.csv /content/drive/MyDrive/wifi_0
!mv /content/5da958dd46f8266d0737457b_0_test.csv /content/drive/MyDrive/wifi_0
!mv /content/5dbc1d84c1eb61796cf7c010_0_test.csv /content/drive/MyDrive/wifi_0
!mv /content/5dc8cea7659e181adb076a3f_0_test.csv /content/drive/MyDrive/wifi_0